In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from scipy import stats
import seaborn as sns
import subprocess
import numpy as np
import gzip

## 1. (2023/08) Single SNP model
(Based on Eric's suggestion)

According to the predixcan paper (Gamazon, et al. 2015)
```
We built prediction models in the DGN whole-blood cohort using LASSO, elastic net (α = 0.5) and polygenic score at several P-value thresholds (single top SNP, 1 × 10−4, 0.001, 0.01, 0.05, 0.5 and 1). We assessed predictive performance using tenfold cross-validation (R2 of estimated GReX versus observed expression) in the initial data set as well as in an independent set
```

```
In the simple polygenic score approach, we estimate wk as the single-variant coefficient derived from regressing the gene expression trait Y on variant Xk (as implemented in the eQTL analysis software Matrix eQTL46) using the reference transcriptome data.
```

### 1.1 Calcaulte PRS using the single top SNP (by p value)

In [2]:
# ############# Helper functions #############
def find_match(lst_dosage, ref, alt):
    '''
    Find exact match for a multiallelic sites.Match by ref and alt allele
    Params:
    - lst_dosage: a list of SNPs located by tabix. Find SNP from this list matching by ref and alt allele
    - alt, ref: position, alt and ref alleles of the SNP to be found
    '''
    for val in lst_dosage:
        _, _, _, cur_ref, cur_alt, _ = val.split(maxsplit=5)
        if cur_ref==ref and cur_alt==alt:
            return val
    return None

def calculate_prs(gwas_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/output/traininig_set_lipid_species_GWAS/adj_for_sex_age_pval_1e-3',
                  dosage_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/subset_vcfs/train/bgziped_dosage',
                  output_fn = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/training/single_top_snp_PRS.txt',
                  rank=0):
    '''
    Calcualte single top SNP PRS
    Params:
    - gwas_output_dir: Directory of GWAS results to get top SNP
    - dosage_output_dir: directory to bgzipped and tabixed dosage files
    - output_fn: Output file to store PRS
    - rank: by default get the SNP with the smallet p value (rank=0). If rank =1 will check the SNP with 2nd smallest p val, and so on for rank=3, 4, 5...
    '''
    output_fh = open(output_fn, 'w')
    output_fh.write('LIPID\tCHR\tPOS\tBETA\t')

    # Write sample IDs from dosage file
    with gzip.open(os.path.join(dosage_output_dir, 'species_chr1.vcf.dosage.gz'), 'rt') as tmp_fh:
        sample_ids = tmp_fh.readline().strip().split()[9:]
        output_fh.write('\t'.join(sample_ids)+'\n')

    # Create PRS from top SNP (beta*dosage)
    count = 0
    for fn in os.listdir(gwas_output_dir):
        lipid = fn.split('_SNPs_pval_0.001.txt')[0]
        # Load SNPs
        try:
            flag = True
            while flag:
                df = pd.read_csv(os.path.join(gwas_output_dir, fn), sep='\t').sort_values(by='P')
                chr_num, _, pos, alt, ref, _, _, beta, _, _ = df.head(rank+1).tail(1).values[0]
                # Get dosage from dosage file
                dosage_fn = os.path.join(dosage_output_dir, f'species_chr{chr_num}.vcf.dosage.gz')
                cmd = f'tabix {dosage_fn} chr{chr_num}:{pos}-{pos}'
                
                # Check for multiallelic sites
                dosage = subprocess.run(cmd.split(), capture_output=True, text=True).stdout.strip().split('\n')
                if len(dosage)>1: # Multiallelic site found
                    dosage = find_match(dosage, ref, alt)
                    if dosage is None: # If SNP not found
                        print('Exact match cannot be found:', )
                        raise Exception('Exact match of SNP cannot be found in dosage file:', f'{chr_num}:{pos}:{ref}:{alt}')
                dosage = np.array([float(x) for x in dosage[0].split()[9:]])
                
                if len(dosage) == 0: # If the SNP was not found with tabix (due to removal of multiallelic sites)
                    rank += 1
                else:
                    flag = False
            
            # dosage = np.array([float(x) for x in subprocess.run(cmd.split(), capture_output=True, text=True).stdout.strip().split()[9:]])
            prs = beta * dosage
            output_fh.write(f'{lipid}\t{chr_num}\t{pos}\t{beta}\t'+'\t'.join([str(x) for x in prs])+'\n')
            output_fh.flush() # Write to output
            count += 1
            print(f'\rProcessing {count}/830', end='', flush=True)
        except:
            print(f'\n# - {fn} can not be loaded')
            
    output_fh.close()
    print('\n#DONE\n')


In [5]:
# Calcualte PRS in training set
print('# Calculate PRS on traning set')
calculate_prs(gwas_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/output/traininig_set_lipid_species_GWAS/adj_for_sex_age_pval_1e-3',
              dosage_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/subset_vcfs/train/bgziped_dosage',
              output_fn = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/training/single_top_snp_PRS.train.txt')

# Calcaulte PRS in test set
print('\n# Calculate PRS on test set')
calculate_prs(gwas_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/output/traininig_set_lipid_species_GWAS/adj_for_sex_age_pval_1e-3',
              dosage_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/subset_vcfs/test/bgziped_dosage',
              output_fn = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/test/single_top_snp_PRS.test.txt')



Processing 76/830
# - all_SNPs_combined_no_dup_no_multiallelic_species.txt can not be loaded
Processing 830/830
#DONE

Processing 76/830
# - all_SNPs_combined_no_dup_no_multiallelic_species.txt can not be loaded
Processing 830/830
#DONE



### 1.2 Run OLS regression using PRS
Model: lipid (residuals, as used in EN and lasso models) ~ PRS

In [7]:
print('# Load trainig PRS')
fn_prs_train = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/training/single_top_snp_PRS.train.txt'
df_prs_train = pd.read_csv(fn_prs_train, sep='\t')
display(df_prs_train.head(2))

print('# Load training data (lipid residuals)')
fn_train = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/lipid_traits_residuals/train/lipid_species_residuals_adj_for_sex_age_pc1-5.txt.reformatted'
df_train = pd.read_csv(fn_train, sep='\t')
display(df_train.head(2))

print('# Load test PRS')
fn_prs_test = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/test/single_top_snp_PRS.test.txt'
df_prs_test = pd.read_csv(fn_prs_test, sep='\t')
display(df_prs_test.head(2))

# Load trainig PRS


,LIPID,CHR,POS,BETA,HD0280_HA0023,BD2180_BD6180,BD2179_BD6179,BD2287_BD6287,HD0119_HD4119,HD0107_HD4107,...,HD0275_HA0018,BD1533_BD5533,BD2455_BD6455,LD0144_LD4144,BD2188_BD6188,BD2606_BD6606,LD0082_LD4082,BD2833_BD6833,BD3346_BA0352,HD0145_HD4145
0,PE-P-18:0-20:3-_-a-,11,61820833,-0.419367,-0.419367,-0.419367,-0.419367,-0.419367,-0.000000,-0.838734,...,-0.838315,-0.830766,-0.419367,-0.838734,-0.419367,-0.830766,-0.838734,-0.41769,-0.822798,-0.827830
1,TG-O-50:2-_[NL-18:2],22,43928975,0.268922,0.000000,0.268922,0.000269,0.000000,0.529238,0.000000,...,0.268922,0.510952,0.268922,0.268922,0.537844,0.536230,0.268922,0.00000,0.268922,0.268922


# Load training data (lipid residuals)


,Sample ID,Sph(d18:1),Sph(d18:2),S1P(d16:1),S1P(d18:0),S1P(d18:1),S1P(d18:2),dhCer(d18:0/16:0),dhCer(d18:0/18:0),dhCer(d18:0/20:0),...,TG(O-54:4) [NL-18:2],Ubiquinone,CE(18:2) [+OH],CE(20:4) [+OH],LPC(18:2) [+OH],LPC(20:4) [+OH],LPC(22:6) [+OH],PC(34:2) [+OH],PC(36:4) [+OH],PC(38:6) [+OH]
0,HA0023,0.310532,0.132709,-0.795436,0.376324,-0.184328,-0.015216,-1.221387,-0.099012,-1.099445,...,-1.850333,-2.189684,-0.622139,-2.174139,-0.410192,-0.732501,-0.867125,-0.667963,-1.420573,-1.373217
1,BD6180,-0.585136,-0.112758,-0.530853,-0.098670,-0.027778,-0.200709,-0.262060,-0.902043,-1.031974,...,-0.237156,-0.347881,-0.008578,-0.235905,-0.439945,-0.825717,-1.095246,0.161639,-0.439407,0.068960


# Load test PRS


,LIPID,CHR,POS,BETA,BD0010_BD4010,BD0382_BD4382,BD0550_BD4550,BD0859_BD4859,BD1229_10Y0282,BD1506_BD5506,...,LD0642_LA0359,LD0653_LA0368,LD0670_LA0385,LD0678_LA0392,LD0683_LA0396,LD0684_LA0397,LD0698_LA0410,LD0701_LA0411,LD0702_LA0412,LD0705_LA0415
0,PE-P-18:0-20:3-_-a-,11,61820833,-0.419367,-0.835798,-0.837057,-0.000000,-0.419367,-0.418948,-0.000000,...,-0.418948,-0.000000,-0.419367,-0.730118,-0.838734,-0.0,-0.419367,-0.418528,-0.000000,-0.000000
1,TG-O-50:2-_[NL-18:2],22,43928975,0.268922,0.267308,0.266771,0.233962,0.005110,0.268922,0.268922,...,0.268922,0.000538,0.000000,0.268922,0.265695,0.0,0.000000,0.537844,0.537844,0.268922


In [8]:
# Run OLS, and calcaulte predicted values on test set 
count = 0
# Save predicted values of test set
output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/test'
output_fn = 'pred_lipid_species.test.txt'
fh_output = open(os.path.join(output_dir, output_fn), 'w')
fh_output.write('LIPID\t' + '\t'.join(df_prs_test.columns[4:]) + '\n')
for lipid in df_train.columns[1:]:
    # lipid = 'Sph(d18:1)'
    mask_train = df_prs_train['LIPID']==lipid.replace('(', '-').replace(')', '-').replace('/', '-').replace(' ', '_')
    X_train = df_prs_train[mask_train].values[0][4:].reshape(-1, 1)
    y_train = df_train[lipid].values
    reg = LinearRegression().fit(X_train, y_train)
    
    try:
        mask_test = df_prs_test['LIPID']==lipid.replace('(', '-').replace(')', '-').replace('/', '-').replace(' ', '_')
        X_test = df_prs_test[mask_test].values[0][4:].reshape(-1, 1)
        pred_vals = reg.predict(X_test)
        fh_output.write(lipid+'\t')
        fh_output.write('\t'.join([str(x) for x in pred_vals]) + '\n')
        count += 1
        print(f'\rProcessing {count}/830', end='', flush=True)
    except:
        # Some SNPs are missing from dosage files to multiallelic sites
        print(f'\n# {lipid} cannot be predicted')
    
fh_output.close()
print('\n# DONE')

Processing 830/830
# DONE


## 2. Top SNPs by p value model
Try PRS regression with more SNPs (pval 5e-8)

### 2.1 Calcaulte PRS using the single top SNP (by p value)
Done in terminal

In [3]:
# ############# Helper functions #############
def find_match(lst_dosage, ref, alt):
    '''
    Find exact match for a multiallelic sites.Match by ref and alt allele
    Params:
    - lst_dosage: a list of SNPs located by tabix. Find SNP from this list matching by ref and alt allele
    - alt, ref: position, alt and ref alleles of the SNP to be found
    '''
    for val in lst_dosage:
        _, _, _, cur_ref, cur_alt, _ = val.split(maxsplit=5)
        if cur_ref==ref and cur_alt==alt:
            return val
    return None

def calcaulte_prs_single_snp(chr_num, pos, gwas_beta, dosage_fn):
    '''
    Calcuate PRS of a single SNP as: PRS = GWAAS_beta * dosage
    Params:
    - chr_num, pos: chromosome and position of the SNP
    - gwas_beta: beta of current SNP
    - dosage_fn: bgzipped and tabixed dosage file (in vcf format)
    Return:
    - list of PRS based on the given SNP
    '''
    cmd = f'tabix {dosage_fn} chr{chr_num}:{pos}-{pos}'
                
    # Check for multiallelic sites and ignore them
    dosage = subprocess.run(cmd.split(), capture_output=True, text=True).stdout.strip().split('\n')
    if len(dosage)>1: # Multiallelic site found
        return np.array([])

    dosage = np.array([float(x) for x in dosage[0].split()[9:]])

    # If the SNP was not found with tabix (Could happen due to removal of multiallelic sites from the processed dosage file)
    # Ignore multiallelic site
    if len(dosage) == 0:
        return np.array([])
    
    prs = gwas_beta * dosage
    return prs
    
    
def calculate_prs_multiple_snps(gwas_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/output/traininig_set_lipid_species_GWAS/adj_for_sex_age_pval_1e-3',
                                gwas_output = '',
                                dosage_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/subset_vcfs/train/bgziped_dosage',
                                output_fn = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/training/multiple_snp_PRS.txt',
                                pval_threshold=1e-4):
    '''
    Calcualte PRS from a list of SNPs (by p value threshold)
    Params:
    - gwas_output_dir: Directory of GWAS results to get top SNPs. Will check all files in this directory if gwas_output is empty
    - gwas_output: (optional) A file containing one GWAS output files perline to parse (just file names, not full path+file name)
    - dosage_output_dir: directory to bgzipped and tabixed dosage files
    - output_fn: Output file to store PRS
    - pval_threshold: filter SNPs by this threshold
    '''
    output_fh = open(output_fn, 'w')
    output_fh.write('LIPID\tCHR\tPOS\tBETA\tN_SNPs\t')

    # Write sample IDs from dosage file
    with gzip.open(os.path.join(dosage_output_dir, 'species_chr1.vcf.dosage.gz'), 'rt') as tmp_fh:
        sample_ids = tmp_fh.readline().strip().split()[9:]
        output_fh.write('\t'.join(sample_ids)+'\n')

    # Create PRS from list of SNP (beta*dosage) filtered by p value threshold
    count = 0
    if gwas_output == '':
        gwas_files = os.listdir(gwas_output_dir)
    else: # If gwas_files is a file containing file names
        gwas_files = []
        with open(gwas_output) as fh:
            line = fh.readline().strip()
            while line != '':
                gwas_files.append(line)
                line = fh.readline().strip()
        
    for fn in gwas_files:
        lipid = fn.split('_SNPs_pval_0.001.txt')[0]
        # Load SNPs, store single SNP PRS in a numpy array
        all_prs = ''
        num_snps = 0
        try:
            df = pd.read_csv(os.path.join(gwas_output_dir, fn), sep='\t')
            mask = df['P']<=pval_threshold
            for i in range(len(df[mask])):
                chr_num, _, pos, alt, ref, _, _, beta, _, _ = df.iloc[i, :].values
                
                # Get dosage from dosage file
                dosage_fn = os.path.join(dosage_output_dir, f'species_chr{chr_num}.vcf.dosage.gz')
                cur_prs = calcaulte_prs_single_snp(chr_num, pos, beta, dosage_fn)
                
                if len(cur_prs) == 0: # If current SNP cannot be found or is a multiallelic site, move on to the next one
                    continue
                
                if len(all_prs)==0:
                    all_prs = cur_prs # If this is the first file
                else:
                    all_prs += cur_prs
                num_snps += 1
                print(f'\rLoading SNPs of #{count} {lipid}: {num_snps}/{len(df[mask])}                     ', flush=True, end='')
            
            prs = all_prs/num_snps
            output_fh.write(f'{lipid}\tNA\tNA\tNA\t{num_snps}\t'+'\t'.join([str(x) for x in prs])+'\n')
            output_fh.flush() # Write to output
            count += 1
            # print(f'\rProcessing {count}/830', end='', flush=True)
        except:
            print(f'\n# - {fn} can not be loaded')
            
    output_fh.close()
    print('\n#DONE\n')


In [ ]:
# Calculate PRS in training set
print('# Calculate PRS on traning set')
calculate_prs_multiple_snps(gwas_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/output/traininig_set_lipid_species_GWAS/adj_for_sex_age_pval_1e-3',
                            gwas_output = '',
                            dosage_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/subset_vcfs/train/bgziped_dosage',
                            output_fn = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/training/multiple_snp_PRS_pval_1e-4.train.txt',
                            pval_threshold = 1e-4)

# Calculate PRS in test set
print('\n# Calculate PRS on test set')
calculate_prs_multiple_snps(gwas_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/output/traininig_set_lipid_species_GWAS/adj_for_sex_age_pval_1e-3',
                            gwas_output = '',
                            dosage_output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/subset_vcfs/test/bgziped_dosage',
                            output_fn = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/test/multiple_snp_PRS_pval_1e-4.test.txt',
                            pval_threshold = 1e-4)



# Calculate PRS on traning set
Loading SNPs of #25 LPC-19:1-_-b-: 2519/2872                                     

### 2.1 Run OLS regression using PRS

In [4]:

fn_prs_train = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/training/multiple_snp_PRS_pval_1e-4.train.txt'
df_prs_train = pd.read_csv(fn_prs_train, sep='\t')
print('# Load trainig PRS:', df_prs_train.shape)
display(df_prs_train.head(2))

fn_train = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/lipid_traits_residuals/train/lipid_species_residuals_adj_for_sex_age_pc1-5.txt.reformatted'
df_train = pd.read_csv(fn_train, sep='\t')
print('# Load training data (lipid residuals):', df_train.shape)
display(df_train.head(2))

fn_prs_test = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/test/multiple_snp_PRS_pval_1e-4.test.txt'
df_prs_test = pd.read_csv(fn_prs_test, sep='\t')
print('# Load test PRS:', df_prs_test.shape)
display(df_prs_test.head(2))

# Load trainig PRS: (830, 1612)


,LIPID,CHR,POS,BETA,N_SNPs,HD0280_HA0023,BD2180_BD6180,BD2179_BD6179,BD2287_BD6287,HD0119_HD4119,...,HD0275_HA0018,BD1533_BD5533,BD2455_BD6455,LD0144_LD4144,BD2188_BD6188,BD2606_BD6606,LD0082_LD4082,BD2833_BD6833,BD3346_BA0352,HD0145_HD4145
0,AC-10:0-,NaN,NaN,NaN,3655,-0.027810,-0.014011,-0.020305,-0.018498,-0.025513,...,-0.018319,-0.022646,-0.015551,-0.023693,-0.014154,-0.023991,-0.015888,-0.021451,-0.010140,-0.016750
1,AC-12:0-,NaN,NaN,NaN,3373,-0.021182,-0.018953,-0.023621,-0.019482,-0.021468,...,-0.020279,-0.021390,-0.021738,-0.021564,-0.025038,-0.022913,-0.013230,-0.017843,-0.018344,-0.018504


# Load training data (lipid residuals): (1607, 831)


,Sample ID,Sph(d18:1),Sph(d18:2),S1P(d16:1),S1P(d18:0),S1P(d18:1),S1P(d18:2),dhCer(d18:0/16:0),dhCer(d18:0/18:0),dhCer(d18:0/20:0),...,TG(O-54:4) [NL-18:2],Ubiquinone,CE(18:2) [+OH],CE(20:4) [+OH],LPC(18:2) [+OH],LPC(20:4) [+OH],LPC(22:6) [+OH],PC(34:2) [+OH],PC(36:4) [+OH],PC(38:6) [+OH]
0,HA0023,0.310532,0.132709,-0.795436,0.376324,-0.184328,-0.015216,-1.221387,-0.099012,-1.099445,...,-1.850333,-2.189684,-0.622139,-2.174139,-0.410192,-0.732501,-0.867125,-0.667963,-1.420573,-1.373217
1,BD6180,-0.585136,-0.112758,-0.530853,-0.098670,-0.027778,-0.200709,-0.262060,-0.902043,-1.031974,...,-0.237156,-0.347881,-0.008578,-0.235905,-0.439945,-0.825717,-1.095246,0.161639,-0.439407,0.068960


# Load test PRS: (830, 687)


,LIPID,CHR,POS,BETA,N_SNPs,BD0010_BD4010,BD0382_BD4382,BD0550_BD4550,BD0859_BD4859,BD1229_10Y0282,...,LD0642_LA0359,LD0653_LA0368,LD0670_LA0385,LD0678_LA0392,LD0683_LA0396,LD0684_LA0397,LD0698_LA0410,LD0701_LA0411,LD0702_LA0412,LD0705_LA0415
0,AC-10:0-,NaN,NaN,NaN,3655,-0.018010,-0.011838,-0.014375,-0.027342,-0.027172,...,-0.029736,-0.027057,-0.012219,-0.020061,-0.030922,-0.013907,-0.015809,-0.016951,-0.011062,-0.014436
1,AC-12:0-,NaN,NaN,NaN,3373,-0.024346,-0.020384,-0.018056,-0.021661,-0.022017,...,-0.014310,-0.020045,-0.023536,-0.021877,-0.022919,-0.020362,-0.026992,-0.014131,-0.015159,-0.013950


In [7]:
# Run OLS, and calcaulte predicted values on test set 
count = 0
# Save predicted values of test set
output_dir = '/data100t1/home/wanying/CCHC/lipidomics/prediction_models/prs_ols/test'
output_fn = 'pred_lipid_species_multiple_snp_1e-4.test.txt'
fh_output = open(os.path.join(output_dir, output_fn), 'w')
fh_output.write('LIPID\t' + '\t'.join(df_prs_test.columns[5:]) + '\n')
for lipid in df_train.columns[1:]:
    # lipid = 'Sph(d18:1)'
    mask_train = df_prs_train['LIPID']==lipid.replace('(', '-').replace(')', '-').replace('/', '-').replace(' ', '_')
    X_train = df_prs_train[mask_train].values[0][5:].reshape(-1, 1)
    y_train = df_train[lipid].values
    reg = LinearRegression().fit(X_train, y_train)
    
    try:
        mask_test = df_prs_test['LIPID']==lipid.replace('(', '-').replace(')', '-').replace('/', '-').replace(' ', '_')
        X_test = df_prs_test[mask_test].values[0][5:].reshape(-1, 1)
        pred_vals = reg.predict(X_test)
        fh_output.write(lipid+'\t')
        fh_output.write('\t'.join([str(x) for x in pred_vals]) + '\n')
        count += 1
        print(f'\rProcessing {count}/830', end='', flush=True)
    except:
        # Some SNPs are missing from dosage files to multiallelic sites
        print(f'\n# {lipid} cannot be predicted')
    
fh_output.close()
print('\n# DONE')

Processing 830/830
# DONE
